# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [66]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('DisasterData', engine)
X = df['message']  #['id','message','original','genre']
Y = df.drop(['id','message','original','genre'], axis = 1)

In [50]:
Y['related'].value_counts()

1.0    19914
0.0     6101
2.0      192
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [33]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]"," ", text)
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")

    #clean_words = []
    #for word in words:
        #clean_words.append(lemmatizer.lemmatize(word).lower().strip())
    clean_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
                           
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

> Found that there are null values, which is why the fit pipeline cell below was not working. As an intial step (may go back and improve on later), let's just drop these few ~100 rows where there are nan values. This makes sense because if we are trying to predict the values of for these columns based of the message, and all these values are Null, they don't give us anything in terms of information


In [35]:
df_clean = df[np.isfinite(df['related'])]
X = df_clean['message']  #['id','message','original','genre']
Y = df_clean.drop(['id','message','original','genre'], axis = 1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [37]:
y_pred = pipeline.predict(X_test)

In [42]:
# y_pred is not a data frame, let's make it into a data frame so we can see things more easily
df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
df_ypred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# Printing out "precision", "recall", and "f1_scores" for each column
for col in y_test.columns:
    print(col)
    print(classification_report(y_test[col].values, df_ypred[col]))

related
             precision    recall  f1-score   support

        0.0       0.29      0.13      0.18      1496
        1.0       0.77      0.91      0.83      5005
        2.0       0.16      0.06      0.09        51

avg / total       0.66      0.72      0.68      6552

request
             precision    recall  f1-score   support

        0.0       0.83      0.97      0.90      5385
        1.0       0.42      0.10      0.16      1167

avg / total       0.76      0.81      0.76      6552

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6527
        1.0       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6552

aid_related
             precision    recall  f1-score   support

        0.0       0.59      0.81      0.68      3765
        1.0       0.48      0.23      0.31      2787

avg / total       0.54      0.57      0.52      6552

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe82aad5598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state

### Using the parameters listed above I can see what parameters yeild the best results. In order to do this, I first structured the code into a model similar to the model shown in the lectures. This involved building a model and having a main() function call that loads the data, fits it, and then shows us the best parameters.

>The starting accuracy was about <b>92.3%</b> so we are hoping to achieve something a little bit higher than this through some trial and error, as well as following the hints listed below

In [101]:
# Function to build the pipeline and create grid search
def build_model():
    
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))    
    ])
    
    # specify parameters for grid search
    parameters = {'vectorizer__max_df': (0.4, .8),
                  'vectorizer__max_features': [None],
                  'tfidf__use_idf': [False],
                  'vectorizer__lowercase': [True],
                  'clf__estimator__n_estimators': [1, 5, 50]
                 }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)
    
    return cv

In [102]:
# Function to read in the data and return X and y to be used for the training and test sets
def load_data():
    engine = create_engine('sqlite:///DisasterData.db')
    df = pd.read_sql_table('DisasterData', engine)
    df_clean = df[np.isfinite(df['related'])]
    X = df_clean['message']  #['id','message','original','genre']
    Y = df_clean.drop(['id','message','original','genre'], axis = 1)
    return X, Y

# Function used by the main function to display the data in a readable way
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    y_pred = y_pred.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    accuracy = (y_pred == y_test).mean().mean()

    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

# Main function call. Process the data and displays results
def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
    
    display_results(model, y_test, df_ypred)


In [ ]:
# This function takes a while to run. Added a 'verbose = 2' so that we can track the progress of the main
# function as it is running. As I tweak parameters I will continue to come back here and run this main function call
main()

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  19.2s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.0s remaining:    0.0s


[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  18.9s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.4, vectorizer__max_features=None, total=  18.8s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None, total=  19.0s
[CV] clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max_df=0.8, vectorizer__max_features=None 
[CV]  clf__estimator__n_estimators=1, tfidf__use_idf=False, vectorizer__lowercase=True, vectorizer__max

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 43.0min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

> I answer this in the main function call above. By adding in the `accuracy` in `display_results` we see how the results improved from 92.3%

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

* Noticed that for the first column, the possible values are 0, 1, and a few 2's. What does this mean here and what is the best way to handle the 2's. Are they errors/should be 1's??

In [ ]:
#Y['related']=Y['related'].map(lambda x: 1 if x == 2 else x)

### 9. Export your model as a pickle file

In [ ]:
# saving
file_name = 'disasterresponse.sav'
pickle.dump(model, open(file_name, 'wb'))
 

In [ ]:
# Try and load the saved model to 
saved_model = pickle.load(open(filename, 'rb'))
findings = saved_model.score(X_test, y_test)
print(findings)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.